In [11]:
%cd ..
%rm -rf no-time-to-backprop

/content


In [12]:
!ls
!nvidia-smi

sample_data
Thu Dec 18 01:46:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P0             32W /   70W |     110MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------

In [13]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.80"

!git clone https://github.com/Francis2002/no-time-to-backprop.git
%cd no-time-to-backprop

Cloning into 'no-time-to-backprop'...
remote: Enumerating objects: 207, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 207 (delta 43), reused 46 (delta 22), pack-reused 138 (from 1)
Receiving objects: 100% (207/207), 88.27 MiB | 32.87 MiB/s, done.
Resolving deltas: 100% (112/112), done.
/content/no-time-to-backprop


In [14]:
!python -V
!ls

Python 3.12.12
constraints.txt			 requirements-macos-cpu.txt  run_on_gpu.ipynb
hpt.py				 requirements.mac.txt	     run_toy_task.py
plotting_scripts		 requirements.txt	     test_rec.py
requirements-base.txt		 run_all_benchmarks.sh	     tgap
requirements.from_linux_env.txt  run_all.sh
requirements-linux-cuda.txt	 run_all_toy.sh


In [15]:
%pip install -r requirements-linux-cuda.txt

In [16]:
import jax
import jaxlib
import sys

print(f"jax: {jax.__version__}")
print(f"jaxlib: {jaxlib.__version__}")
print(f"executable: {sys.executable}")
print(f"devices: {jax.devices()}")


jax: 0.8.1
jaxlib: 0.8.1
executable: /usr/bin/python3
devices: [CudaDevice(id=0)]


In [7]:
!python3 run_toy_task.py -m ONLINE -a ZUC --dataset mooc --task link_classification

[*] Results will be stored in /content/no-time-to-backprop/results_for_rotational_vs_none/mooc_ONLINE
[*] d_model set to 64
Running iteration: 0
[*] Output file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists. Using existing file.
object address  : 0x7bd2cbad2f20
object refcount : 3
object type     : 0xa2a4e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [17]:
# Here we run experiments
!python3 run_toy_task.py -m ONLINE -a ZUC --num_layers 4 --num_hidden 128 --activation full_glu --decoder MLP --mixing rotational_full --prenorm --encoder --layer_output --extra_skip --dataset mooc --task link_classification --batching_strategy rearranged --lr 0.001 --dropout 0.1 --weight_decay 0.001 --batch_size 50 --num_epochs 50 --num_gradient_accumulation_steps 16 --dedupe --lr_schedule warmup_cosine

[*] Results will be stored in /content/no-time-to-backprop/results_for_rotational_vs_none/mooc_ONLINE
[*] d_model set to 64
Running iteration: 0
[*] Output file /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz already exists. Using existing file.
W1218 01:49:04.772052   14419 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1218 01:49:04.774689   14400 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
[*] Loaded data from /content/no-time-to-backprop/tgap/data/npzs/mooc_stream.npz with 7047 nodes
[*] Trainable Parameters: 302337
[*] Model running on device: cuda:0
All params tree leaf keys:
|- cell
|  |- params
|  |  |- decoder
|  |  |  |- bias  shape=(1,)
|  |  |  |- kernel  shape=(64, 1)
|  |  |- encoder
|  |     |- encoder
|  |     |  |- bias  shape=(64,)
| 